In [ ]:
pip install openai==0.28

In [1]:
import openai
import ipywidgets as widgets
from IPython.display import display

# Inicializar la clave API como None
api_key = None

# Función para sugerir títulos
def reescribir_nota(texto):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"reescribe el siguiente texto con formato de nota, tu rol es de un periodista especializado , la escritura tiene que ser entendible para un publico joven,  El título y el subtítulo deben tener un puntaje SEO alto. Luego crea una breve reflexión: {texto}"}
        ]
    )
    leng_joven = response.choices[0]['message']['content'].split('\n')
    return [nota_rewrite.strip() for nota_rewrite in leng_joven if nota_rewrite.strip()]

# Función para sugerir tags
def sugerir_tags(texto):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"Busca las palabras mas relevantes para usarlas como tags: {texto}"}
        ]
    )
    tags = response.choices[0]['message']['content']
    return tags.split(", ")

# Función para generar un resumen breve (menor a 400 caracteres)
def generar_resumen(texto):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"Resume el siguiente texto en menos de 400 caracteres: {texto}"}
        ]
    )
    resumen = response.choices[0]['message']['content']
    return resumen[:400]  # Asegurar que el resumen no exceda los 1000 caracteres

# Función para generar imagen usando DALL-E con estilos adicionales
def generar_imagen(bajada, estilos=[]):
    prompt = bajada
    if estilos:
        prompt += "Tiene que ser una imagen de alta calidad, super realista, con los textos claros  " + " que tenga colores acordes a la imagen, que sea una imagen expresiva, que el lector entienda que esa imagen es del texto leido  ".join(estilos)
    
    response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="512x512"  # Tamaño de la imagen
    )
    return response['data'][0]['url']

# Función para generar bajada
def generar_bajada(texto):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"Genera una bajada breve en 150 palabras, para este texto: {texto}"}
        ]
    )
    return response.choices[0]['message']['content']

# Función para generar información extra
def Traducir_a_ingles(texto):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"Traduce el texto a idioma ingles USA: {texto}"}
        ]
    )
    return response.choices[0]['message']['content']


# Función para manejar el evento de botón
def procesar_texto(b):
    global api_key
    if api_key is None:
        output_area.append_stdout("Por favor, ingresa tu API key de OpenAI.\n")
        return

    openai.api_key = api_key  # Establecer la clave API
    
    texto = texto_input.value
    if not texto.strip():
        output_area.append_stdout("Por favor, ingresa un texto.\n")
        return
    
    opcion = opcion_selector.value
    
    if opcion == 1:
        leng_joven = reescribir_nota(texto)
        output_area.append_stdout("\nReescribir nota con perfil Joven:\n")
        for nota_rewrite in leng_joven:
            output_area.append_stdout(nota_rewrite + "\n")
    
    elif opcion == 2:
        resumen = generar_resumen(texto)
        output_area.append_stdout(f"Explicacion para la imagen: {resumen}\n")
        imagen_url = generar_imagen(resumen)
        output_area.append_stdout(f"\nURL de la imagen generada: {imagen_url}\n")
    
    elif opcion == 3:
        tags = sugerir_tags(texto)
        output_area.append_stdout("\nSugerencias de tags:\n")
        output_area.append_stdout(", ".join(tags) + "\n")
    
    elif opcion == 4:
        bajada = generar_bajada(texto)
        output_area.append_stdout(f"\nBajada (Resumen): {bajada}\n")
    
    elif opcion == 5:
        traslate = Traducir_a_ingles(texto)
        output_area.append_stdout(f"\nTraducir a Ingles: {traslate}\n")
    

# Función para manejar el evento de ingreso de API key
def set_api_key(b):
    global api_key
    api_key = api_key_input.value.strip()  # Guardar la API key
    output_area.append_stdout("API key establecida. Que comience la magia!.\n")

# Widgets para la interfaz gráfica
api_key_input = widgets.Password(  # Cambiado a Password
    placeholder='Ingresa tu API key aquí...',
    description='API Key:',
    layout=widgets.Layout(width='90%')
)

texto_input = widgets.Textarea(
    placeholder='Ingresa tu texto aquí...',
    description='Texto:',
    layout=widgets.Layout(width='90%', height='150px')
)

opcion_selector = widgets.Dropdown(
    options=[
        ('Reescritura de nota', 1),
        ('Generar Imagen', 2),
        ('Sugerir Tags', 3),
        ('Generar Resumen', 4),
        ('Traducir a Ingles ', 5),
    ],
    description='Selectores:',
)


# Botón con estilo

boton_set_api_key = widgets.Button(
    description="Registrar API Key",
    button_style="success", 
    style={'button_color': 'green'} 
)



boton_procesar = widgets.Button(
    description="Enviar",
    button_style="success",
    style={'button_color': 'blue'}
)
output_area = widgets.Output()

# Conectar los botones a las funciones
boton_set_api_key.on_click(set_api_key)
boton_procesar.on_click(procesar_texto)

# Mostrar los widgets
display(api_key_input, boton_set_api_key, texto_input, opcion_selector, boton_procesar, output_area)


Password(description='API Key:', layout=Layout(width='90%'), placeholder='Ingresa tu API key aquí...')

Button(button_style='success', description='Registrar API Key', style=ButtonStyle(button_color='green'))

Textarea(value='', description='Texto:', layout=Layout(height='150px', width='90%'), placeholder='Ingresa tu t…

Dropdown(description='Selectores:', options=(('Reescritura de nota', 1), ('Generar Imagen', 2), ('Sugerir Tags…

Button(button_style='success', description='Enviar', style=ButtonStyle(button_color='blue'))

Output()